In [1]:
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import shapely
from scipy.spatial.distance import cdist

In [2]:
class KMeans(object):
    def __init__(self, k=8, euclid = True):
        self.k = k
        if (euclid):
            self._distance = 'euclidean'
        else:
            self._distance = self._distance_haversine
    
    def _step(self):
        """Compute distance, assign groups, recompute centers"""
        distance = cdist(self.X,self.cluster_centers,metric=self._distance)
        self.labels = distance.argmin(1)
       # centers = np.zeros((self.k,2))
        for cluster in range(self.k):
            points = self.X[self.labels == cluster]
            if len(points) == 0:
                distance = cdist(self.X,self.cluster_centers,metric=self._distance)
                mean_dist = np.mean(distance,0)
                self.cluster_centers[cluster] = mean_dist.argmax()
            else:
                self.cluster_centers[cluster] = np.mean(points,0)
       # self.cluster_centers = centers
        
    def _distance_haversine(self,a,b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        d_lat = lat_2 - lat_1
        d_lon = lon_2 - lon_1
        
        arc = np.sin(d_lat/2.0)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(d_lon/2)**2
        
        c = 2 * np.arcsin(np.sqrt(arc))
        km = 6372.8 * c
        return km
    
    def _init_centers(self, X):
        unique = np.unique(X, axis=0)
        index = np.random.permutation(len(unique))[:self.k]
        return unique[index]
    
    def fit(self,X, centers = None):
        '''Expects centers to be inputted, if not random'''
        self.labels = np.zeros(len(X))
        self.X = X
        if centers is not None:
            self.cluster_centers = centers 
        else:
            self.cluster_centers = self._init_centers(X)
        old_centers = np.zeros((self.k,2))
    #    self.i = 0
        while(not np.array_equal(old_centers, self.cluster_centers)):
            old_centers = self.cluster_centers.copy()
            self._step()
         #   self.i+=1

In [30]:
from shapely.geometry import Point
from geopandas import GeoDataFrame

demographics = gpd.read_file('./census.geoJSON')

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def point_similarity(X,geo_labels, euc_labels,k):
    '''For an inputted series of points, geodesic labels, euclidean labels, and k-value
       returns the point-similarity index per geodesic cluster
    '''

    euc_cluster_totals = np.zeros(k,dtype=np.int)
    geo_euc_composition = [np.zeros(k,dtype=np.int)* 1 for i in range(k)]
    
    for index,point in enumerate(geo_labels):
        euc_cluster_totals[euc_labels[index]] += 1
        geo_euc_composition[point][euc_labels[index]] += 1
    
    point_sim = []
    for geo_cluster in range(k):
        sim = 0
        for euc_cluster in range(k):
            matching_points = geo_euc_composition[geo_cluster][euc_cluster]
            euc_percentage = matching_points / euc_cluster_totals[euc_cluster]
            geo_percentage = matching_points / np.sum(geo_euc_composition[geo_cluster])
            sim += euc_percentage * geo_percentage
        point_sim.append(sim)

    return np.array(point_sim)

def minority_probability(X,cluster_number,geo_labels,demographics):
        points = X[geo_labels == cluster_number]
        # geoJSON puts points in Long/Lat order
        # but points are in lat/long earlier
        hull = shapely.geometry.multipoint.MultiPoint([[p[1],p[0]] for p in points]).convex_hull
  
        pop = np.zeros(7)
        for index in range(len(demographics)):
            census_tract = demographics.loc[index,'geometry']
            intersect = hull.intersection(census_tract)
            overlap = intersect.area/census_tract.area
            if (overlap != 0):
                pop = pop + (np.array(demographics.loc[index,['White','Black or African American', 'American Indian and Ala Native',
                   'Asian','Native Hawaiian/other Pac Isl', 'Multiple Race',
                   'Other Race']]) * overlap)
        
        if (np.all(pop ==0)):
            return 0
        
        return (pop[1:]/np.sum(pop)).sum()

def bias_index(X, geo_labels, euc_labels, demographics, k):

    dissimilarity_index = 1 - point_similarity(X,geo_labels,euc_labels,k)
    minority_prob = np.array([minority_probability(X,cluster,geo_labels,demographics) 
                              for cluster in range(k)])
    
    potential_bias = minority_prob * dissimilarity_index
    return potential_bias.mean()

    



In [31]:
bias_index(X,geodesic.labels,euclid.labels,demographics,k)
geodesic.labels == euclid.labels

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
import warnings
holes = open('./experiment_heap/holes.txt','r')
temp = holes.read().splitlines()
holes.close()
temp = np.random.permutation(temp)
for item in temp:
    file = item.split('-')[0] + '/' + item.split('-')[1]
    k = int(item.split('-')[2])
    
    df = pd.read_csv('../data/' + file, sep =';')
        
    x = []
    y = []

    df['Points'] = df['Location'].apply(gen_coords)
    points = [Point(xy) for xy in zip(x,y)]
    crs = {'init': 'epsg:4326'}
    geo_df = GeoDataFrame(df,crs=crs, geometry=points)
    theft_both = geo_df.copy()

    test_list = []

    for index in range(len(theft_both)):
        test_list.append(df.loc[index, 'Points'])

    X = np.array(test_list)
   
    print(file)
    

    bias_data = f'{file};{k};'
    warnings.simplefilter('error')
    success = 0
    iteration = 0
    while(success < 49):
        try:
            iteration +=1
            euclid = KMeans(k=k)
            geodesic = KMeans(k=k, euclid = 'false')
            centers = euclid._init_centers(X)

            euclid.fit(X,centers=centers)
            geodesic.fit(X,centers=centers)
          #  print(euclid.labels)
          #  print(geodesic.labels)

            bias = bias_index(X,geodesic.labels,euclid.labels,demographics,k)
            print(bias)
            bias_data += str(bias) + ';'
            success +=1
            print(success)
        except Warning:
            if (iteration > 200 and success == 0):
                break
            continue
    if success == 0:
        continue
    folder = item.split('-')[0]
    file_name = item.split('-')[1]
    file_name = file_name.split('.csv')[0]
    with open(f'./experiment_heap/{folder}_{file_name}_{k}.csv','w') as f:
        print(f'./experiment_heap/{folder}_{file_name}_{k}.csv')
        f.write(bias_data)


data_2014/m_theft_july.csv
0.0
1
0.0
2
0.0
3
0.0
4
0.0
5
0.0
6
0.0
7
0.0
8
0.0
9
0.0
10
0.0
11
0.0
12
0.0
13
0.0
14
0.0
15
0.0
16
0.0
17
0.0
18
0.0
19
0.0
20
0.0
21
0.0
22
0.0
23
0.0
24
